In [6]:
import os
try:
    import flexiblecc as fcc
except ImportError as e:
    !pip install flexiblecc
    import flexiblecc as fcc

if not os.path.exists("../../CalImgs/"):
    if not os.path.exists("Flexible-Camera-Calibration/CalImgs"):
        !git clone https://github.com/SimonLBSoerensen/Flexible-Camera-Calibration 
    datasetpath = "Flexible-Camera-Calibration/CalImgs/cam_0*"
else:
    datasetpath = "../../CalImgs/cam_0*"

In [8]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [9]:
color_cal_imgs_files = glob.glob(datasetpath)
color_cal_imgs = np.array([cv2.imread(f) for f in tqdm(color_cal_imgs_files)])
gray_cal_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in tqdm(color_cal_imgs)]

100%|██████████| 94/94 [00:00<00:00, 2191.88it/s]


In [10]:
pattern_size = (12, 12)

In [11]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewErrors, points_3D, points_2D = fcc.OldSchoolCC.calibrate_camera(gray_cal_imgs, pattern_size, verbose=1)

100%|██████████| 94/94 [00:11<00:00,  8.51it/s]
Doing camera calibrate


In [13]:
print(img_points.shape)

(94, 144, 1, 2)
